In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [2]:
train_path = './Dataset_2/Train/'
test_path = 'Dataset_2/Test/'
data_dir = './Dataset_2'

train_lable = 'Dataset_2/Train.csv'
test_lable = 'Dataset_2/Test.csv'

In [3]:
NUM_CATEGORIES = len(os.listdir(train_path))
NAME_CATEGORIES = os.listdir(train_path)
print(NUM_CATEGORIES)
print(NAME_CATEGORIES)

43
['12', '16', '19', '32', '28', '30', '40', '24', '4', '2', '14', '38', '9', '20', '3', '41', '7', '0', '17', '39', '34', '22', '33', '8', '42', '1', '5', '27', '31', '26', '10', '25', '35', '6', '36', '15', '37', '23', '21', '18', '11', '29', '13']


In [4]:
# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

In [5]:
# move resize
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            #列表不够整齐，如果创建多维列表，子列表中的元素得个数一致，如果不一致，那么默认还是一维列表，把子列表当做元素而已
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))      
            imgray = cv2.cvtColor(np.array(resize_image), cv2.COLOR_BGR2GRAY)
            flat_arr=imgray.ravel()
            # image_data.append(np.array(imgray))
            image_data.append(flat_arr)
            image_labels.append(i)
        except:
            print("Error in " + img)

In [6]:
img_dict={"images":np.array(imgray),"data":np.array(image_data),"target":np.array(image_labels)}

In [7]:
img_dict['data']
train = img_dict['data']/255

In [8]:
img_dict['target']

array([ 0,  0,  0, ..., 42, 42, 42])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train, img_dict['target'], test_size=0.3, random_state=0, shuffle=True)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(27446, 900)
(11763, 900)
(27446,)
(11763,)


In [11]:
X_train

array([[0.1372549 , 0.12941176, 0.12941176, ..., 0.12156863, 0.11764706,
        0.11372549],
       [0.23137255, 0.15294118, 0.1254902 , ..., 0.05882353, 0.05490196,
        0.07843137],
       [0.10196078, 0.09803922, 0.09411765, ..., 0.09019608, 0.09411765,
        0.09411765],
       ...,
       [0.06666667, 0.07058824, 0.11372549, ..., 0.18039216, 0.17254902,
        0.16862745],
       [0.23137255, 0.22745098, 0.25490196, ..., 0.40392157, 0.4       ,
        0.39215686],
       [0.05490196, 0.05098039, 0.05490196, ..., 0.05098039, 0.05490196,
        0.05490196]])

# GaussianNB

In [12]:
gnb = GaussianNB().fit(X_train,y_train)

In [13]:
score = gnb.score(X_test,y_test)
print('准确率：',score)

准确率： 0.12173765195953413


In [14]:
y_pre = gnb.predict(X_test) #查看预测结果
y_pre

array([34, 31,  6, ..., 32,  8,  6])

In [15]:
print(y_pre == y_test)

[False False False ... False False False]


In [16]:
prob = gnb.predict_proba(X_test) # 查看预测的概率结果
prob #每个样本在每个标签下的概率结果

array([[3.81340617e-228, 4.31093302e-195, 0.00000000e+000, ...,
        2.11138640e-273, 0.00000000e+000, 0.00000000e+000],
       [2.50452721e-217, 7.38117599e-158, 9.95930883e-098, ...,
        3.01282027e-135, 2.37456977e-102, 4.05291385e-093],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 7.98448695e-230],
       ...,
       [4.96784757e-256, 1.81955005e-155, 3.34154450e-083, ...,
        1.51620325e-164, 2.33667619e-104, 0.00000000e+000],
       [8.69855929e-318, 6.06797940e-201, 3.49413295e-134, ...,
        8.49704032e-173, 2.86715634e-141, 1.25939016e-080],
       [0.00000000e+000, 0.00000000e+000, 9.98070062e-276, ...,
        1.09551909e-280, 6.77061930e-288, 8.21210678e-071]])

In [17]:
# 使用混淆矩阵查看贝叶斯的分类结果
from sklearn.metrics import confusion_matrix as CM
CM(y_test,y_pre)

array([[ 37,   1,   4, ...,   0,   1,   0],
       [137,  98,  38, ...,   0,  15,   0],
       [ 27, 117,  63, ...,   0,   9,   0],
       ...,
       [  3,   8,   5, ...,   2,   4,   0],
       [  4,   5,   0, ...,   0,  26,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

# KNeighborsClassifier

In [18]:
n_neighbors = [1,3,5,7,9]
for k in n_neighbors:
    knn=KNeighborsClassifier(n_neighbors=k)###构建邻居值为1的knn分类器
    knn.fit(X_train,y_train)###训练模型
    prediction=knn.predict(X_test)###对测试集进行预测
    print("k = {}, acc = {}".format(k,accuracy_score(y_test,prediction)))

k = 1, acc = 0.9060613788999405
k = 3, acc = 0.860154722434753
k = 5, acc = 0.8301453710788065
k = 7, acc = 0.8030264388336309
k = 9, acc = 0.7806681968885488


# SVM

In [19]:
clf = SVC(kernel="linear",random_state=0)
clf.fit(X_train, y_train)#训练
y_predict = clf.predict(X_test)#预测
print(accuracy_score(y_test, y_predict))#评分

0.9380260137719969


# randomforest


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

dtc = DecisionTreeClassifier(random_state=0)
rfc = RandomForestClassifier(random_state=0)
dtc = dtc.fit(X_train,y_train)
rfc = rfc.fit(X_train,y_train)
score_c = clf.score(X_test,y_test)
score_r = rfc.score(X_test,y_test)
print(score)

# AFTER PCA

In [26]:
pca=PCA(n_components=100,whiten=True,random_state=0).fit(X_train)
X_train_pca=pca.transform(X_train)
X_test_pca=pca.transform(X_test)
knn=KNeighborsClassifier(n_neighbors=10)###构建邻居值为1的knn分类器
knn.fit(X_train_pca,y_train)
knn_predict=knn.predict(X_test_pca)###对测试集进行预测
print(accuracy_score(y_test,knn_predict))

0.8066819688854884


In [21]:
pca=PCA(n_components=100,whiten=True,random_state=0).fit(X_train)
X_train_pca=pca.transform(X_train)
X_test_pca=pca.transform(X_test)
clf=SVC(kernel="linear",random_state=0)
clf.fit(X_train_pca, y_train)#训练
clf_predict=clf.predict(X_test_pca)
print(accuracy_score(y_test,clf_predict))

0.9172830060358752


In [22]:
components_pca=pd.DataFrame(columns=["n_components","SVC_score","knn_score"])
for i in range(100):
    pca=PCA(n_components=i+1,whiten=True,random_state=0).fit(X_train)
    X_train_pca=pca.transform(X_train)
    X_test_pca=pca.transform(X_test)
    clf=SVC(kernel="linear",random_state=0)
    clf.fit(X_train_pca, y_train)
    clf_predict=clf.predict(X_test_pca)
    knn=KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train_pca,y_train)
    knn_predict=knn.predict(X_test_pca)
    components_pca=components_pca.append([{"n_components":i+1,"SVC_score":accuracy_score(y_test,clf_predict),"knn_score":accuracy_score(y_test,knn_predict)}], ignore_index=True)

In [23]:
components_pca

,n_components,SVC_score,knn_score
0,1,0.086458,0.070730
1,2,0.093939,0.203860
2,3,0.097934,0.361727
3,4,0.103970,0.430842
4,5,0.111366,0.505313
...,...,...,...
95,96,0.915413,0.809572
96,97,0.916348,0.808467
97,98,0.914478,0.806512
98,99,0.916858,0.807617


In [24]:
components_pca[components_pca['knn_score']==components_pca['knn_score'].max()]

,n_components,SVC_score,knn_score
91,92,0.915838,0.810593


In [25]:
components_pca[components_pca['SVC_score']==components_pca['SVC_score'].max()]

,n_components,SVC_score,knn_score
99,100,0.917283,0.806682
